## Анализ поведения пользователей и эффективности каналов их привлечения (сервис доставки продуктов)

Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

Описание данных:

date – дата совершения события  
event - событие  
  app_install – установка приложения  
  app_start – открыть приложения   
  register – зарегистрироваться   
  search – перейти на страницу поиска товаров (каталог)  
  open_item – открыть товар  
  choose_item – отправить товар в корзину  
  tap_basket – перейти в корзину  
  purchase – подтверждение покупки  
gender – пол пользователя  
os_name – платформа пользователя  
city – город пользователя  
device_id – идентификатор устройства пользователя  
urm_source – канал, с которого пришел пользователь  
  yandex-direct – Яндекс директ  
  google_ads – реклама в Google  
  vk_ads – реклама в ВК  
  instagram_ads – реклама в instagram  
  facebook_ads – реклама в facebook  
    referal – акция «приведи друга»  
Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы    
purchase_sum – стоимость покупки (при совершении события ‘purchase’)  

Обратите внимание на следующее:

- В выгрузке только уникальные действия пользователей за каждый день 
- Можно миновать стадию установки приложения, если оно было установлено ранее
- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

In [1]:
# Импортируем библиотеки

import pandas as pd

In [2]:
# Считываем данные

df = pd.read_csv('KC_case_data.csv')
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [3]:
# Проверим тип данных

df.dtypes

date             object
event            object
purchase_sum    float64
os_name          object
device_id         int64
gender           object
city             object
utm_source       object
dtype: object

In [4]:
# Изменим тип данных для столбца с датой

df['date'] = pd.to_datetime(df.date)

In [5]:
df.dtypes

date            datetime64[ns]
event                   object
purchase_sum           float64
os_name                 object
device_id                int64
gender                  object
city                    object
utm_source              object
dtype: object

### Посчитайте MAU

In [6]:
# Выделим месяц из даты

df['month'] = df.date.dt.month

In [7]:
# Сгруппируем данные помесячно и посчитаем MAU

df.groupby('month', as_index = False) \
  .agg({'device_id' : 'nunique'})

,month,device_id
0,1,99161
1,2,75032
2,3,74623


### Посчитайте количество установок в январе

In [8]:
df.query('month == 1 & event == "app_install"').shape[0]

80297

### Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей?
Примечание: считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

In [9]:
# Определяем для каждого device_id дату первой установки

install = df.query('event == "app_install"').groupby('device_id', as_index = False) \
    .agg({'date' : 'min'}) \
    .rename(columns = {'date' : 'date_install'})

install.head()

,device_id,date_install
0,4013,2020-01-15
1,4016,2020-01-04
2,4046,2020-01-04
3,4065,2020-03-19
4,4067,2020-01-31


In [10]:
# считаем кол-во юзеров, установивших приложение, для каждой даты установки 

install_users = install.groupby('date_install', as_index = False) \
                        .agg({'device_id' : 'count'}) \
                        .rename(columns = {'device_id' : 'install_users'})
install_users.head()

,date_install,install_users
0,2020-01-01,3579
1,2020-01-02,3144
2,2020-01-03,2402
3,2020-01-04,1831
4,2020-01-05,1671


In [11]:
# Определяем для каждого device_id дату первой покупки

first_purchase = df.query('event == "purchase"').groupby('device_id', as_index = False) \
    .agg({'date' : 'min'}) \
    .rename(columns = {'date' : 'date_first_purchase'})

first_purchase.head()

,device_id,date_first_purchase
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4120,2020-01-07
4,4125,2020-01-07


In [12]:
# объединяем для каждого device_id дату первой установки и дату первой покупки

cr_users = install.merge(first_purchase, on = 'device_id', how = 'inner')
cr_users.head()

,device_id,date_install,date_first_purchase
0,4046,2020-01-04,2020-01-04
1,4120,2020-01-07,2020-01-07
2,4125,2020-01-07,2020-01-07
3,4202,2020-01-08,2020-02-27
4,4234,2020-02-19,2020-02-19


In [13]:
# считаем разницу в днях между датой первой покупки и датой первой установки

cr_users['days_bw_purchase'] = (cr_users.date_first_purchase - cr_users.date_install).dt.days
cr_users.head()

,device_id,date_install,date_first_purchase,days_bw_purchase
0,4046,2020-01-04,2020-01-04,0
1,4120,2020-01-07,2020-01-07,0
2,4125,2020-01-07,2020-01-07,0
3,4202,2020-01-08,2020-02-27,50
4,4234,2020-02-19,2020-02-19,0


In [14]:
cr_users.days_bw_purchase.min()

0

In [15]:
cr_users.days_bw_purchase.max()

89

In [16]:
# отбираем только юзеров с разницей в днях <=7 (сконвертированные юзеры) и группируем их по дате первой установки

converted_users = cr_users.query('days_bw_purchase <= 7') \
                          .groupby('date_install', as_index = False) \
                          .agg({'device_id' : 'count'}) \
                          .rename(columns = {'device_id' : 'converted_users'})
converted_users.head()

,date_install,converted_users
0,2020-01-01,1408
1,2020-01-02,1186
2,2020-01-03,834
3,2020-01-04,639
4,2020-01-05,587


In [17]:
# объединяем кол-во юзеров, установивших приложение, и кол-во сконвертированных юзеров для каждой даты первой установки

cr = install_users.merge(converted_users, on = 'date_install', how = 'inner')
cr.head()

,date_install,install_users,converted_users
0,2020-01-01,3579,1408
1,2020-01-02,3144,1186
2,2020-01-03,2402,834
3,2020-01-04,1831,639
4,2020-01-05,1671,587


In [18]:
# считаем конверсию

cr['cr'] = (cr.converted_users / cr.install_users * 100).round(1)
cr.head()

,date_install,install_users,converted_users,cr
0,2020-01-01,3579,1408,39.3
1,2020-01-02,3144,1186,37.7
2,2020-01-03,2402,834,34.7
3,2020-01-04,1831,639,34.9
4,2020-01-05,1671,587,35.1


In [19]:
# Находим когорту с максимальной конверсией

cr.query('cr == cr.max()').date_install

0   2020-01-01
Name: date_install, dtype: datetime64[ns]

### Посчитайте количество пользователей в разрезе маркетинговых каналов

In [20]:
df.groupby('utm_source', as_index = False) \
  .agg({'device_id' : 'nunique'}) \
  .sort_values('device_id', ascending = False)

,utm_source,device_id
0,-,52273
6,yandex-direct,40712
2,google_ads,38096
5,vk_ads,34086
3,instagram_ads,31048
1,facebook_ads,25959
4,referal,15926


### Посчитайте конверсию зарегистрированных клиентов на всех этапах воронки

In [21]:
# Определим дату регистрации каждого device_id

register_date = df.query('event == "register"').groupby('device_id', as_index = False).agg({'date' : 'min'}) \
                    .rename(columns = {'date' : 'register_date'})
register_date.head()

,device_id,register_date
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4067,2020-01-31
4,4120,2020-01-07


In [22]:
# добавим столбец с датой регистрации в общую таблицу, оставим только тех, у кого есть дата регистрации (через Inner)

register_users = df.merge(register_date, on = 'device_id', how = 'inner')
register_users.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,register_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,1,2020-01-07
1,2020-01-01,search,NaN,android,669460,female,Moscow,-,1,2020-01-07
2,2020-01-03,app_start,NaN,android,669460,female,Moscow,-,1,2020-01-07
3,2020-01-03,choose_item,NaN,android,669460,female,Moscow,-,1,2020-01-07
4,2020-01-03,search,NaN,android,669460,female,Moscow,-,1,2020-01-07


In [23]:
# кол-во открывших приложение зарегистрированных пользователей
# (зарегистрированный = дата регистрации < даты события)

register = register_users.query('event == "app_start" & register_date < date').device_id.nunique()
register

40991

In [24]:
# кол-во перешедших в поиск зарегистрированных пользователей

searh = register_users.query('event == "search" & register_date < date').device_id.nunique()
searh

40482

In [25]:
# конверсия из открытия в поиск

searh / register * 100

98.75826400917275

In [26]:
# кол-во зарегистрированных пользователей, добавивших товар в корзину

add_users = register_users.query('event == "choose_item" & register_date < date').device_id.nunique()
add_users

37926

In [27]:
# конверсия из поиска в добавление товара

add_users / searh * 100

93.68608270342375

In [28]:
# кол-во зарегистрированных пользователей, перешедших в корзину

basket = register_users.query('event == "tap_basket" & register_date < date').device_id.nunique()
basket

34517

In [29]:
# конверсия из добавления товара в переход в корзину

basket/add_users*100

91.01144333702473

### Посчитайте конверсию в первую покупку в разрезе каналов привлечения

In [30]:
# Посчитаем кол-во юзеров, впервые совершивших покупку, в разбивке по каналам привлечения

purchase =  df.query('event == "purchase"') \
              .groupby('utm_source', as_index = False) \
              .agg({'device_id' : 'nunique'}) \
              .rename(columns = {'device_id' : 'purchase_users'})
purchase.head()

,utm_source,purchase_users
0,-,16598
1,facebook_ads,9017
2,google_ads,11339
3,instagram_ads,10762
4,referal,6362


In [31]:
# Посчитаем кол-во юзеров, впервые открывших приложение, в разбивке по каналам привлечения

open_users = df.query('event == "app_start"') \
               .groupby('utm_source', as_index = False) \
               .agg({'device_id' : 'nunique'}) \
               .rename(columns = {'device_id' : 'open_users'})
open_users.head()

,utm_source,open_users
0,-,52268
1,facebook_ads,25957
2,google_ads,38092
3,instagram_ads,31045
4,referal,15924


In [32]:
# Объединим таблицы и посчитаем конверсию, отсортируем по возрастанию

cr_purchase = open_users.merge(purchase, on = 'utm_source', how = 'inner')
cr_purchase['cr'] = cr_purchase.purchase_users / cr_purchase.open_users * 100
cr_purchase.sort_values('cr')

,utm_source,open_users,purchase_users,cr
6,yandex-direct,40707,12028,29.547744
2,google_ads,38092,11339,29.767405
0,-,52268,16598,31.755567
3,instagram_ads,31045,10762,34.665808
1,facebook_ads,25957,9017,34.738221
5,vk_ads,34079,12364,36.280407
4,referal,15924,6362,39.952273


## Посчитайте медианный первый чек пользователей в разрезе маркетинговых каналов (учитываются только первые покупки пользователей)

In [33]:
df.query('purchase_sum.isna() == False') \
  .groupby(['utm_source', 'device_id'], as_index = False) \
  .agg({'date' : 'min', 'purchase_sum' : 'sum'}) \
  .groupby('utm_source', as_index = False) \
  .agg({'purchase_sum' : 'median'}) \
  .sort_values('purchase_sum', ascending = False)

,utm_source,purchase_sum
4,referal,530.0
3,instagram_ads,507.0
1,facebook_ads,504.5
0,-,499.5
5,vk_ads,499.5
2,google_ads,467.0
6,yandex-direct,465.5


## Посчитайте ROMI по каналам привлечения

Данные по затратам на рекламу:  
Яндекс – 10 491 707 руб.  
Гугл – 10 534 878 руб.  
Фейсбук – 8 590 498 руб.  
Инстаграм – 8 561 626 руб.  
ВК – 9 553 531руб.  
Расходы на реферальную программу - 3 972 400 руб.

In [34]:
# ROMI = (Валовая прибыль - Маркетинговые расходы) / Маркетинговые расходы

In [35]:
# Посчитаем стоимость покупки по каналам привлечения

purchase_sum = df.query('purchase_sum.isna() == False') \
                 .groupby('utm_source', as_index = False) \
                 .agg({'purchase_sum' : 'sum'})
purchase_sum

,utm_source,purchase_sum
0,-,21449749.5
1,facebook_ads,12249901.0
2,google_ads,12868276.0
3,instagram_ads,14546969.0
4,referal,8837044.5
5,vk_ads,16389652.5
6,yandex-direct,13915368.0


In [36]:
# Создадим таблицу с маркетинговыми расходами

marketing = pd.DataFrame({'utm_source' : ['-', 'facebook_ads', 'google_ads', 'instagram_ads', 'referal', 'vk_ads', 'yandex-direct'], 
                          'marketing_expenses' : [0,  8590498, 10534878, 8561626, 3972400, 9553531, 10491707]})
marketing

,utm_source,marketing_expenses
0,-,0
1,facebook_ads,8590498
2,google_ads,10534878
3,instagram_ads,8561626
4,referal,3972400
5,vk_ads,9553531
6,yandex-direct,10491707


In [37]:
# Объединим таблицы

romi = purchase_sum.merge(marketing, on = 'utm_source', how = 'inner')
romi

,utm_source,purchase_sum,marketing_expenses
0,-,21449749.5,0
1,facebook_ads,12249901.0,8590498
2,google_ads,12868276.0,10534878
3,instagram_ads,14546969.0,8561626
4,referal,8837044.5,3972400
5,vk_ads,16389652.5,9553531
6,yandex-direct,13915368.0,10491707


In [38]:
# Посчитаем ROMI

romi['romi'] = (romi.purchase_sum - romi.marketing_expenses) / romi.marketing_expenses * 100
romi.sort_values('romi', ascending = False)

,utm_source,purchase_sum,marketing_expenses,romi
0,-,21449749.5,0,inf
4,referal,8837044.5,3972400,122.461094
5,vk_ads,16389652.5,9553531,71.555967
3,instagram_ads,14546969.0,8561626,69.908952
1,facebook_ads,12249901.0,8590498,42.598264
6,yandex-direct,13915368.0,10491707,32.632068
2,google_ads,12868276.0,10534878,22.149265
